In [3]:
import tensorflow as tf

import keras
from keras import layers
from keras.models import Sequential
from keras.layers import Conv2D, Dense, MaxPooling2D, Flatten


import cv2
import numpy as np
import matplotlib.pyplot as plt

import PIL
import os
from os import listdir

In [4]:
data_dir = "Data_Cancer/"
labels = listdir(data_dir)
labels

['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'vasc']

In [5]:
img_height = 224
img_width = 224
batch_size = 32

##### Changing image sizes to (224, 224)

In [6]:
for folder in listdir(data_dir):
    for image in listdir(data_dir + folder):
        img_path = data_dir+folder+"/"+image
        img = cv2.imread(img_path)
        resized_img = cv2.resize(img, (img_height, img_width))
        cv2.imwrite(img_path, resized_img)

        

##### Creating training, validation dataset

In [11]:
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset = "training",
    seed = 1234,
    image_size = (img_height, img_width),
    batch_size = batch_size
)

validation_dataset = keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset = "validation",
    seed = 1234,
    image_size = (img_height, img_width),
    batch_size = batch_size
)

Found 10015 files belonging to 7 classes.
Using 8012 files for training.


2024-07-13 00:28:24.397909: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-13 00:28:24.510973: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-13 00:28:24.511047: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-13 00:28:24.514469: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-13 00:28:24.514572: I external/local_xla/xla/stream_executor

Found 10015 files belonging to 7 classes.
Using 2003 files for validation.


In [ ]:
train_dataset = train_dataset.map(lambda x,y: (layers.Rescaling(1./255)(x), y))
validation_dataset = validation_dataset.map(lambda x,y: (layers.Rescaling(1./255)(x), y))

In [46]:
input_layer = layers.Input((img_height, img_width, 3))
conv_layer_1 = layers.Conv2D(64, kernel_size = (7,7), strides = (2,2), padding = "same", activation = "relu")(input_layer)
x = layers.MaxPooling2D((2,2))(conv_layer_1)

## ResidualBlock_1
res_block_conv = layers.Conv2D(64, (3,3), strides = 1, padding = "same", activation = "relu")(x)
fx = layers.Conv2D(64, (3,3), strides = 1, padding = "same", activation = "relu")(res_block_conv)
hx = fx + x

x = hx
## ResidualBlock_2
res_block_conv = layers.Conv2D(64, (3,3), strides = 1, padding = "same", activation = "relu")(x)
fx = layers.Conv2D(64, (3,3), strides = 1, padding = "same", activation = "relu")(res_block_conv)
hx = fx + x

x = hx
## ResidualBlock_3
res_block_conv = layers.Conv2D(64, (3,3), strides = 1, padding = "same", activation = "relu")(x)
fx = layers.Conv2D(64, (3,3), strides = 1, padding = "same", activation = "relu")(res_block_conv)
hx = fx + x

x = hx

## Not ResidualBlock 4
res_block_conv = layers.Conv2D(128, (3,3), strides = 2, padding = "same", activation = "relu")(x)
fx = layers.Conv2D(128, (3,3), strides = 1, padding = "same", activation = "relu")(res_block_conv)


x = fx
## ResidualBlock_5
res_block_conv = layers.Conv2D(128, (3,3), strides = 1, padding = "same", activation = "relu")(x)
fx = layers.Conv2D(128, (3,3), strides = 1, padding = "same", activation = "relu")(res_block_conv)
hx = fx + x

x = hx
## ResidualBlock_6
res_block_conv = layers.Conv2D(128, (3,3), strides = 1, padding = "same", activation = "relu")(x)
fx = layers.Conv2D(128, (3,3), strides = 1, padding = "same", activation = "relu")(res_block_conv)
hx = fx + x

x = hx
## ResidualBlock_7
res_block_conv = layers.Conv2D(128, (3,3), strides = 1, padding = "same", activation = "relu")(x)
fx = layers.Conv2D(128, (3,3), strides = 1, padding = "same", activation = "relu")(res_block_conv)
hx = fx + x

x = hx
## Block 8

res_block_conv = layers.Conv2D(256, (3,3), strides = 2, padding = "same", activation = "relu")(x)
fx = layers.Conv2D(256, (3,3), strides = 1, padding = "same", activation = "relu")(res_block_conv)

x = fx
## Block 9
res_block_conv = layers.Conv2D(256, (3,3), strides = 1, padding = "same", activation = "relu")(x)
fx = layers.Conv2D(256, (3,3), strides = 1, padding = "same", activation = "relu")(res_block_conv)
hx = fx + x

x = hx
## Block 10
res_block_conv = layers.Conv2D(256, (3,3), strides = 1, padding = "same", activation = "relu")(x)
fx = layers.Conv2D(256, (3,3), strides = 1, padding = "same", activation = "relu")(res_block_conv)
hx = fx + x

x = hx
## Block 11
res_block_conv = layers.Conv2D(256, (3,3), strides = 1, padding = "same", activation = "relu")(x)
fx = layers.Conv2D(256, (3,3), strides = 1, padding = "same", activation = "relu")(res_block_conv)
hx = fx + x

x = hx
## Block 12
res_block_conv = layers.Conv2D(256, (3,3), strides = 1, padding = "same", activation = "relu")(x)
fx = layers.Conv2D(256, (3,3), strides = 1, padding = "same", activation = "relu")(res_block_conv)
hx = fx + x

x = hx
## Block 13
res_block_conv = layers.Conv2D(256, (3,3), strides = 1, padding = "same", activation = "relu")(x)
fx = layers.Conv2D(256, (3,3), strides = 1, padding = "same", activation = "relu")(res_block_conv)
hx = fx + x

x = hx
## Block 14
res_block_conv = layers.Conv2D(512, (3,3), strides = 2, padding = "same", activation = "relu")(x)
fx = layers.Conv2D(512, (3,3), strides = 1, padding = "same", activation = "relu")(res_block_conv)

x = fx
## Block 15
res_block_conv = layers.Conv2D(512, (3,3), strides = 1, padding = "same", activation = "relu")(x)
fx = layers.Conv2D(512, (3,3), strides = 1, padding = "same", activation = "relu")(res_block_conv)
hx = fx + x

x = hx
## Block 16
res_block_conv = layers.Conv2D(512, (3,3), strides = 1, padding = "same", activation = "relu")(x)
fx = layers.Conv2D(512, (3,3), strides = 1, padding = "same", activation = "relu")(res_block_conv)
hx = fx + x

x = hx

avg_pool = layers.GlobalAveragePooling2D()(x)
fc = layers.Dense(len(labels))(avg_pool)

In [47]:
model = keras.Model(inputs=input_layer, outputs=fc)
model.summary()

Model: "functional_10"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_11      │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_101 (Conv2D) │ (None, 112, 112,  │      9,472 │ input_layer_11[0… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_9     │ (None, 56, 56,    │          0 │ conv2d_101[0][0]  │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_102 (Conv2D) │ (None, 56, 56,    │     36,928 │ max_pooling2d_9[… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_103 (Conv2D) │ (None, 56, 56,    │     36,928 │ conv2d_102[0][0]  │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_34 (Add)        │ (None, 56, 56,    │          0 │ conv2d_103[0][0], │
│                     │ 64)               │            │ max_pooling2d_9[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_104 (Conv2D) │ (None, 56, 56,    │     36,928 │ add_34[0][0]      │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_105 (Conv2D) │ (None, 56, 56,    │     36,928 │ conv2d_104[0][0]  │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_35 (Add)        │ (None, 56, 56,    │          0 │ conv2d_105[0][0], │
│                     │ 64)               │            │ add_34[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_106 (Conv2D) │ (None, 56, 56,    │     36,928 │ add_35[0][0]      │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_107 (Conv2D) │ (None, 56, 56,    │     36,928 │ conv2d_106[0][0]  │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_36 (Add)        │ (None, 56, 56,    │          0 │ conv2d_107[0][0], │
│                     │ 64)               │            │ add_35[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_108 (Conv2D) │ (None, 28, 28,    │     73,856 │ add_36[0][0]      │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_109 (Conv2D) │ (None, 28, 28,    │    147,584 │ conv2d_108[0][0]  │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_110 (Conv2D) │ (None, 28, 28,    │    147,584 │ conv2d_109[0][0]  │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_111 (Conv2D) │ (None, 28, 28,    │    147,584 │ conv2d_110[0][0]  │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_37 (Add)        │ (None, 28, 28,    │          0 │ conv2d_111[0][0]

 Total params: 21,106,823 (80.52 MB)

 Trainable params: 21,106,823 (80.52 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
epochs = 10
model.compile(optimizer = "adam", loss = keras.losses.SparseCategoricalCrossentropy, metrics=["accuracy"])
history = model.fit(train_dataset, validation_data = validation_dataset, epochs = epochs)

Epoch 1/10


I0000 00:00:1720826371.233889   62823 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_2765', 8 bytes spill stores, 8 bytes spill loads



251/251 ━━━━━━━━━━━━━━━━━━━━ 82s 264ms/step - accuracy: 0.0285 - loss: 15.4159 - val_accuracy: 0.0305 - val_loss: 15.5881
Epoch 2/10
251/251 ━━━━━━━━━━━━━━━━━━━━ 54s 213ms/step - accuracy: 0.0290 - loss: 15.4077 - val_accuracy: 0.0305 - val_loss: 15.5881
Epoch 3/10
251/251 ━━━━━━━━━━━━━━━━━━━━ 54s 215ms/step - accuracy: 0.0294 - loss: 15.4026 - val_accuracy: 0.0305 - val_loss: 15.5881
Epoch 4/10
177/251 ━━━━━━━━━━━━━━━━━━━━ 14s 200ms/step - accuracy: 0.0284 - loss: 15.3847